# Creating photonic circuits

Let's start by creating a very simple circuit, in pure python, by connecting a series of components from a PDK.

In [ ]:
import cspdk.si220
from cspdk.si220 import cells
import gdsfactory as gf

cspdk.si220.PDK.activate()

pdk = gf.get_active_pdk()

In [ ]:
c = gf.Component()
s1 = c << cells.straight_sc(length=50)
s2 = c << cells.straight_sc(length=20)
b1 = c << cells.bend_sc()
b1.connect('o1', s1.ports['o2'])
s2.connect('o1', b1.ports['o2'])
c.add_port('o1', port=s1.ports['o1'])
c.add_port('o2', port=s2.ports['o2'])
c.rename('first_circuit')
c

Note that the circuit can now be serialized to a yaml format that captures both the connectivity (netlist) information and placements that would be required to rebuild the circuit.

In [ ]:
yaml_circuit = c.to_yaml()
print(yaml_circuit)

And the yaml netlist can also be deserialized to reinstantiate the gdsfactory component and back out to GDS.

In [ ]:
loaded = gf.read.from_yaml(yaml_circuit)
loaded

Of course, you could also edit the netlist in yaml and regenerate a different circuit. Let's try it.

In [ ]:
# first, let's write the netlist to file
c.write_netlist("first_circuit.pic.yml")
# now try adding another instance to the netlist. maybe another bend at the end

In [ ]:
gf.clear_cache()
gf.read.from_yaml('first_circuit.pic.yml')

## Routes

Until now, our circuits have been fairly simple. For more complicated circuits, we want to be able to easily define routes between components. There are various functions in gdsfactory which allow us to do this.

### Automatic bundle routing
In photonics, it is common to want route a group of waveguides together in what we call a "bundle". This is similar to what is often called a "bus" in electrical routing.

The `get_bundle` function can route a bundle between a list of input and corresponding output ports with a simple automatic routing routine.

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                               [mmi2.ports["o3"], mmi2.ports["o2"]])
for route in bundle:
    c2.add(route.references)
c2

### Custom bundle routing
Sometimes we wish to route in a more custom fashion. Whether it be to route around obstacles, acheive a particular pathlength, etc., the `get_bundle_from_steps` function allows us to route in the sequence of directions specified by our `steps` directives.

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_from_steps([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          steps=[
                                              {"dx": 20},
                                              {"dy": 100},
                                              {"dx": 20}
                                          ])
for route in bundle:
    c2.add(route.references)
c2

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_all_angle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          connector="simple",
                                          )
for route in bundle:
    c2.add(route.references)
c2

In [ ]:
gf.CONF.allow_offgrid = True

c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_all_angle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          connector="simple",
                                          steps=[
                                              {"ds": 20, "exit_angle": -20},
                                              {"ds": 50, "exit_angle": 180 + 20},
                                          ]
                                          )
for route in bundle:
    c2.add(route.references)
c2.write_gds("sample_offgrid.gds", flatten_offgrid_references=True)
c2